<a href="https://colab.research.google.com/github/ymuto0302/RW2025/blob/main/MLP_implementations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 様々な MLP の実装

## 隠れ層が固定されている場合の実装
`nn.Sequential` を利用した実装である。
- メリット： シンプルで理解しやすい
- デメリット： 柔軟性に欠ける（異なる構造には使用できない）

In [11]:
import torch
import torch.nn as nn

# MLPモデルの定義
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, output_size):
        super(MultiLayerPerceptron, self).__init__()

        # レイヤを順次的に接続する
        self.network = nn.Sequential(
            # 入力層
            nn.Linear(input_size, 100),
            # 活性化関数
            nn.ReLU(),
            # 隠れ層
            nn.Linear(100, 50),
            # 活性化関数
            nn.ReLU(),
            # 出力レイヤ
            nn.Linear(50, output_size)
        )

    def forward(self, x):
        return self.network(x)


## 各層を柔軟に制御したい場合の実装
`__init()__` では各層の定義のみ行い，`forward()` で各層の処理を行う実装である。
- メリット： 隠れ層や ReLU関数からの出力を確認しやすい ／ `forward()` 内で複雑な処理を追加可能
- デメリット： 層数が増えると記述量も増える

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F # ReLU活性化関数のため

# MLPモデルの定義
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, output_size):
        super(MultiLayerPerceptron, self).__init__()

        # 第１層目：入力層から第１隠れ層への全結合層
        self.fc1 = nn.Linear(input_size, 100)
        # 第2層目：第1隠れ層から第2隠れ層への全結合層
        self.fc2 = nn.Linear(100, 50)
        # 第3層目：第2隠れ層から出力層への全結合層
        self.fc3 = nn.Linear(50, output_size)

    def forward(self, x):
        # 第１層：入力を第１隠れ層に通し，ReLU活性化を適用
        x = F.relu(self.fc1(x))
        # 第2層：第１隠れ層の出力を第2隠れ層に通し，ReLU活性化を適用
        x = F.relu(self.fc2(x))
        # 第3層：第2隠れ層の出y録を最終出力へ変換
        x = self.fc3(x)

        return x


## 動的構造に対応したい場合の実装
引数で受け取ったパラメータに基づいて，`__init()__` 内でネットワーク構造を構築する実装である。
- メリット： 隠れ層について，任意の層数やサイズ対応できる ／ 構造を動的に変更可能なため，ハイパーパラメータ最適に利用できる
- デメリット： 初学者には分かりにくい

In [1]:
import torch
import torch.nn as nn

# MLPモデルの定義
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, output_size, hidden_layer_sizes=(10,)):
        super(MultiLayerPerceptron, self).__init__()

        # ネットワークの各層を格納するModuleListを初期化
        layers = nn.ModuleList()

        # 各層のサイズを決定するリストを作成
        layer_sizes = [input_size] + list(hidden_layer_sizes) + [output_size]

        # 隣接する層間の線形変換(nn.Linear)と活性化関数(nn.ReLU)を順次追加
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            # 出力層（最後の層）以外は活性化関数を追加
            if i < len(layer_sizes) - 2:
                layers.append(nn.ReLU())

        # ModuleList から nn.Sequential に変換
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


さらに活性化関数も指定したい場合，次のコードとなる。

In [7]:
import torch
import torch.nn as nn

# MLPモデルの定義
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, output_size, hidden_layer_sizes=(10,), activation='relu'):
        super(MultiLayerPerceptron, self).__init__()

        # 活性化関数のリスト
        activate_functions = {
            'relu': nn.ReLU(),
            'sigmoid': nn.Sigmoid(),
            'tanh': nn.Tanh(),
            'leaky_relu': nn.LeakyReLU(),
            'elu': nn.ELU()
        }

        # ネットワークの各層を格納するModuleListを初期化
        layers = nn.ModuleList()

        # 各層のサイズを決定するリストを作成
        layer_sizes = [input_size] + list(hidden_layer_sizes) + [output_size]

        # 隣接する層間の線形変換(nn.Linear)と活性化関数(nn.ReLU)を順次追加
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            # 出力層（最後の層）以外は活性化関数を追加
            if i < len(layer_sizes) - 2:
                layers.append(activate_functions[activation])

        # ModuleList から nn.Sequential に変換
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


---
## 上記クラスの動作チェックのためのコード

In [8]:
# 構造固定版
# mlp = MultiLayerPerceptron(2, 3)

# 動的構造対応版 (activate function 指定なし)
# mlp = MultiLayerPerceptron(2, 3, hidden_layer_sizes=(10,5))

# 動的構造対応版 (activate function 指定あり)
mlp = MultiLayerPerceptron(2, 3, hidden_layer_sizes=(10,5), activation='sigmoid')

mlp

MultiLayerPerceptron(
  (network): Sequential(
    (0): Linear(in_features=2, out_features=10, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=10, out_features=5, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=5, out_features=3, bias=True)
  )
)

In [9]:
x = torch.randn(10, 2) # batch size 10, input_size 2
# x = x.unsqueeze(0)
x.size()

torch.Size([10, 2])

In [10]:
mlp(x)

tensor([[-0.3889,  0.1982, -0.3532],
        [-0.3871,  0.1977, -0.3531],
        [-0.3781,  0.1839, -0.3583],
        [-0.3935,  0.2060, -0.3498],
        [-0.3661,  0.1684, -0.3641],
        [-0.4048,  0.2224, -0.3431],
        [-0.3796,  0.1856, -0.3577],
        [-0.3751,  0.1795, -0.3598],
        [-0.3770,  0.1837, -0.3586],
        [-0.4012,  0.2160, -0.3458]], grad_fn=<AddmmBackward0>)